In [ ]:
from robovast.common.analysis import read_output_files, read_output_csv, calculate_speeds_from_poses, print_bag_topics, for_each_test, get_behavior_info
import pandas as pd

DATA_DIR = ''

try:
    df = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "poses.csv"))
    df.loc[df['frame'] == 'turtlebot4_base_link_gt', 'position.x'] += 8

    # # for debugging
    # for_each_test(DATA_DIR, lambda test_dir: print_bag_topics(test_dir))
    # read complete rosbag
    # df_bag = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "rosbag2.csv"))

    df_behaviors = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "behaviors.csv"))
    df_behavior_info = get_behavior_info('differential_drive_robot.nav_to_pose', df_behaviors)
except Exception as e:
    print(f"Error reading rosbag files: {e}")
    raise SystemExit("No rosbag data found.")

## Overview

In [ ]:
from robovast_nav.gui import MapVisualizer
viz = MapVisualizer()

# Cache filtered dataframes to avoid repeated filtering
df_robot_mask = df['frame'] == 'base_link'
df_gt_mask = df['frame'] == 'turtlebot4_base_link_gt'
path_robot = list(zip(df.loc[df_robot_mask, 'position.x'], df.loc[df_robot_mask, 'position.y']))
path_groundtruth = list(zip(df.loc[df_gt_mask, 'position.x'], df.loc[df_gt_mask, 'position.y']))

viz.load_map("/opt/ros/jazzy/share/nav2_bringup/maps/depot.yaml")
viz.create_figure(figsize=(14, 12))

_ = viz.draw_path(path_groundtruth, linewidth=1, color='blue', label='Ground Truth')
_ = viz.draw_path(path_robot, linewidth=1, color='red', label='Robot Localization')
_ = viz.ax.legend(loc='upper left', fontsize=9, ncol=2)


## Screen Capture

In [ ]:
from IPython.display import HTML
from pathlib import Path
import os

# Display video capture
video_filename = 'capture.webm'
video_path = os.path.join(DATA_DIR, video_filename)

if os.path.exists(video_path):
    # Use HTML for full control over video width
    abs_path = os.path.abspath(video_path)
    file_url = Path(abs_path).as_uri()
    
    html = f'''<video controls loop autoplay muted style="width: 100%; max-width: 100%;">
        <source src="{file_url}" type="video/webm">
        Your browser does not support the video tag.
    </video>'''
    
    display(HTML(html))
else:
    print(f"Video not found at: {video_path}")

## Errors

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Split the dataframe by frame - use boolean indexing once and cache results
df_robot = df[df['frame'] == 'base_link'].copy()
df_groundtruth = df[df['frame'] == 'turtlebot4_base_link_gt'].copy()

# Sort immediately to avoid repeated sorting
df_robot.sort_values('timestamp', inplace=True)
df_robot.reset_index(drop=True, inplace=True)
df_groundtruth.sort_values('timestamp', inplace=True)
df_groundtruth.reset_index(drop=True, inplace=True)


In [ ]:
# Check if dataframes have sufficient data
if len(df_robot) == 0 or len(df_groundtruth) == 0:
    print("Warning: Insufficient data for analysis. Skipping error analysis.")
    print(f"  df_robot rows: {len(df_robot)}")
    print(f"  df_groundtruth rows: {len(df_groundtruth)}")
    raise SystemExit("Insufficient data")

# Find common time range - data already sorted from previous cell
time_start = max(df_robot['timestamp'].iloc[0], df_groundtruth['timestamp'].iloc[0])
time_end = min(df_robot['timestamp'].iloc[-1], df_groundtruth['timestamp'].iloc[-1])

# print(f"Common time range: {time_start:.2f} to {time_end:.2f} seconds")

# Convert to numpy arrays for faster interpolation
gt_timestamps = df_groundtruth['timestamp'].values
gt_pos_x = df_groundtruth['position.x'].values
gt_pos_y = df_groundtruth['position.y'].values

# Create interpolation functions for ground truth
gt_interp_x = interp1d(gt_timestamps, gt_pos_x, kind='linear', fill_value='extrapolate', assume_sorted=True)
gt_interp_y = interp1d(gt_timestamps, gt_pos_y, kind='linear', fill_value='extrapolate', assume_sorted=True)

gt_yaw = df_groundtruth['orientation.yaw'].values
gt_interp_z = interp1d(gt_timestamps, gt_yaw, kind='linear', fill_value='extrapolate', assume_sorted=True)

In [ ]:
# Filter robot data to common time range - use boolean indexing efficiently
time_mask = (df_robot['timestamp'] >= time_start) & (df_robot['timestamp'] <= time_end)
df_robot_aligned = df_robot[time_mask].copy()

# Convert to numpy array for vectorized operations
robot_timestamps = df_robot_aligned['timestamp'].values

# Create relative time (normalized to start at 0)
df_robot_aligned['relative_time'] = robot_timestamps - robot_timestamps[0]

# Interpolate ground truth at robot timestamps - batch operation
df_robot_aligned['gt_x'] = gt_interp_x(robot_timestamps)
df_robot_aligned['gt_y'] = gt_interp_y(robot_timestamps)

df_robot_aligned['gt_z'] = gt_interp_z(robot_timestamps)

In [ ]:
# Calculate position errors - use numpy arrays for speed
robot_x = df_robot_aligned['position.x'].values
robot_y = df_robot_aligned['position.y'].values
gt_x = df_robot_aligned['gt_x'].values
gt_y = df_robot_aligned['gt_y'].values

position_error_x = robot_x - gt_x
position_error_y = robot_y - gt_y
absolute_position_error = np.sqrt(position_error_x**2 + position_error_y**2)

# Calculate orientation error if available
robot_yaw = df_robot_aligned['orientation.yaw'].values
gt_z = df_robot_aligned['gt_z'].values
orientation_error = robot_yaw - gt_z
# Normalize to [-pi, pi] - vectorized
orientation_error = np.arctan2(np.sin(orientation_error), np.cos(orientation_error))

In [ ]:
# Create figure with subplots for errors
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Use numpy arrays directly for plotting (faster than series)
timestamps = df_robot_aligned['timestamp'].values

# 1. Absolute Position Error over time
axes[0, 0].plot(timestamps, absolute_position_error, 'b-', linewidth=1)
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Absolute Position Error (m)')
axes[0, 0].set_title('Absolute Position Error Over Time')
axes[0, 0].grid(True, alpha=0.3)

# 2. Position Error Components (X and Y)
axes[0, 1].plot(timestamps, position_error_x, 'r-', label='X Error', linewidth=1)
axes[0, 1].plot(timestamps, position_error_y, 'g-', label='Y Error', linewidth=1)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Position Error (m)')
axes[0, 1].set_title('Position Error Components')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Absolute Orientation Error over time
if orientation_error is not None:
    axes[1, 0].plot(timestamps, np.abs(orientation_error), 'purple', linewidth=1)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Absolute Orientation Error (rad)')
    axes[1, 0].set_title('Absolute Orientation Error Over Time')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'Orientation data not available', 
                    ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title('Absolute Orientation Error Over Time')

# 4. Relative Position Error (error between consecutive points)
relative_position_error = np.diff(absolute_position_error)
axes[1, 1].plot(timestamps[1:], relative_position_error, 'orange', linewidth=1)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Relative Position Error Change (m)')
axes[1, 1].set_title('Relative Position Error (Rate of Change)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Additional detailed analysis
fig2, axes2 = plt.subplots(1, 2, figsize=(14, 5))

# Pre-calculate statistics to avoid repeated computation
mean_error = absolute_position_error.mean()
median_error = np.median(absolute_position_error)

# Distribution of absolute position errors
axes2[0].hist(absolute_position_error, bins=50, color='blue', alpha=0.7, edgecolor='black')
axes2[0].axvline(mean_error, color='red', linestyle='--', 
                 linewidth=2, label=f'Mean: {mean_error:.3f} m')
axes2[0].axvline(median_error, color='green', linestyle='--', 
                 linewidth=2, label=f'Median: {median_error:.3f} m')
axes2[0].set_xlabel('Absolute Position Error (m)')
axes2[0].set_ylabel('Frequency')
axes2[0].set_title('Distribution of Absolute Position Errors')
axes2[0].legend()
axes2[0].grid(True, alpha=0.3)

# Cumulative distribution - optimized using numpy
sorted_errors = np.sort(absolute_position_error)
n = len(sorted_errors)
cumulative = np.arange(1, n + 1) * (100.0 / n)
axes2[1].plot(sorted_errors, cumulative, 'b-', linewidth=2)
axes2[1].set_xlabel('Absolute Position Error (m)')
axes2[1].set_ylabel('Cumulative Percentage (%)')
axes2[1].set_title('Cumulative Distribution of Position Errors')
axes2[1].grid(True, alpha=0.3)

# Calculate percentiles once
percentiles = [50, 90, 95]
percentile_values = np.percentile(absolute_position_error, percentiles)

# Mark specific percentiles
for percentile, value in zip(percentiles, percentile_values):
    axes2[1].axhline(percentile, color='red', linestyle='--', alpha=0.5)
    axes2[1].axvline(value, color='red', linestyle='--', alpha=0.5)
    axes2[1].text(value, percentile + 2, f'{percentile}%: {value:.3f}m', fontsize=9)

plt.tight_layout()
plt.show()

# Print statistics - use pre-calculated values where possible
std_error = absolute_position_error.std()
max_error = absolute_position_error.max()

print("\n=== Position Error Statistics ===")
print(f"Mean: {mean_error:.4f} m")
print(f"Std Dev: {std_error:.4f} m")
print(f"Median: {median_error:.4f} m")
print(f"95th percentile: {percentile_values[2]:.4f} m")
print(f"Max: {max_error:.4f} m")

if orientation_error is not None:
    abs_orient_error = np.abs(orientation_error)
    mean_abs_orient = abs_orient_error.mean()
    std_orient = orientation_error.std()
    max_abs_orient = abs_orient_error.max()
    
    print("\n=== Orientation Error Statistics ===")
    print(f"Mean absolute: {mean_abs_orient:.4f} rad ({np.degrees(mean_abs_orient):.2f}°)")
    print(f"Std Dev: {std_orient:.4f} rad ({np.degrees(std_orient):.2f}°)")
    print(f"Max absolute: {max_abs_orient:.4f} rad ({np.degrees(max_abs_orient):.2f}°)")


## Speed Analysis

In [ ]:
df_gt_speeds = calculate_speeds_from_poses(df_groundtruth)

In [ ]:
# Create crisp speed visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

timestamps_gt = df_gt_speeds['timestamp'].values

# 1. Linear Speed Distribution
axes[0].hist(df_gt_speeds['linear_speed'].values, bins=40, color='steelblue', 
                edgecolor='black', linewidth=0.8, alpha=0.7)
mean_linear = df_gt_speeds['linear_speed'].mean()
median_linear = df_gt_speeds['linear_speed'].median()
axes[0].axvline(mean_linear, color='red', linestyle='--', linewidth=1.5, label=f'Mean: {mean_linear:.3f} m/s')
axes[0].axvline(median_linear, color='green', linestyle='--', linewidth=1.5, label=f'Median: {median_linear:.3f} m/s')
axes[0].set_xlabel('Linear Speed (m/s)', fontsize=10)
axes[0].set_ylabel('Frequency', fontsize=10)
axes[0].set_title('Linear Speed Distribution', fontsize=11, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3, linewidth=0.5, axis='y')

# 2. Angular Speed Distribution
axes[1].hist(df_gt_speeds['angular_speed'].values, bins=40, color='coral', 
                edgecolor='black', linewidth=0.8, alpha=0.7)
mean_angular = df_gt_speeds['angular_speed'].mean()
median_angular = df_gt_speeds['angular_speed'].median()
axes[1].axvline(mean_angular, color='red', linestyle='--', linewidth=1.5, label=f'Mean: {mean_angular:.3f} rad/s')
axes[1].axvline(median_angular, color='green', linestyle='--', linewidth=1.5, label=f'Median: {median_angular:.3f} rad/s')
axes[1].axvline(0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('Angular Speed (rad/s)', fontsize=10)
axes[1].set_ylabel('Frequency', fontsize=10)
axes[1].set_title('Angular Speed Distribution', fontsize=11, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3, linewidth=0.5, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Combined speed profile visualization
fig, ax = plt.subplots(figsize=(14, 5))

# Create twin axis for angular speed
ax2 = ax.twinx()

# Plot linear speed on primary axis
line1 = ax.plot(timestamps_gt, df_gt_speeds['linear_speed'].values, 
                'b-', linewidth=1.5, label='Linear Speed', alpha=0.8)

# Plot angular speed on secondary axis
line2 = ax2.plot(timestamps_gt, df_gt_speeds['angular_speed'].values, 
                 'r-', linewidth=1.5, label='Angular Speed', alpha=0.8)

# Configure axes
ax.set_xlabel('Time (s)', fontsize=11, fontweight='bold')
ax.set_ylabel('Linear Speed (m/s)', fontsize=11, fontweight='bold', color='b')
ax2.set_ylabel('Angular Speed (rad/s)', fontsize=11, fontweight='bold', color='r')
ax.set_title('Combined Speed Profile', fontsize=12, fontweight='bold')

# Style the axes
ax.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='r')
ax.grid(True, alpha=0.3, linewidth=0.5)
ax.set_ylim(bottom=0)

# Add legend
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
print(f"Ground truth speed statistics:")
print(f"Linear speed - Mean: {df_gt_speeds['linear_speed'].mean():.3f} m/s, Max: {df_gt_speeds['linear_speed'].max():.3f} m/s")
print(f"Angular speed - Mean abs: {np.abs(df_gt_speeds['angular_speed']).mean():.3f} rad/s, Max abs: {np.abs(df_gt_speeds['angular_speed']).max():.3f} rad/s")